In [ ]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(file)
# print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

In [12]:
input_file_li=[r"FR_output_features.csv", r"IT_output_features.csv", r"SK_output_features.csv"]
session_li=["main0", "main25", "main50", "main75"]

In [74]:
import re
import random
import pandas as pd

In [99]:
session='main0'
feature='NHR'
input_df=pd.read_csv(input_file_li[0])
print("input_df[:5]", input_df[:5])

input_df[:5]    Unnamed: 0  Mean Pitch   Max Pitch  Mean Intensity  Max Intensity  \
0           0   62.378338  179.198877       69.949677      78.114851   
1           1   51.702229  185.547010       69.422789      79.892876   
2           2   53.932833  511.778945       69.036319      77.949600   
3           3   52.723395  194.008268       69.591178      78.351791   
4           4   42.930457  169.834422       67.006694      77.372524   

     Jitter   Shimmer        NHR  dyad  speaker    session  sentence  \
0  0.019850  0.060315  18.430881  6162       61  imitation        10   
1  0.021415  0.068678  17.854420  6162       61  imitation        11   
2  0.024644  0.063827  16.671792  6162       61  imitation        12   
3  0.018365  0.054014  19.276305  6162       61  imitation        13   
4  0.025257  0.092803  16.287774  6162       61  imitation        14   

   duration(s)  size(K)                          file_name  
0         7.67   660.94  SK0102_1_imitation555_10_mono.wav  

In [104]:
session_input_df=input_df[input_df['session']==session]

dyad_reg=r"(\d{2})(\d{2})"
dyad_li=list(set(session_input_df['dyad'].tolist()))

for dyad in dyad_li:
    print("processing", dyad)
    match = re.search(dyad_reg, str(dyad))
    speaker=int(match.group(1))
    partner=int(match.group(2))

    session_speaker_df=session_input_df[session_input_df['speaker']==speaker]
    session_partner_df=session_input_df[session_input_df['speaker']==partner]

    speaker_sent_li=session_speaker_df['sentence'].tolist()
    partner_sent_li=session_partner_df['sentence'].tolist()
    
    speaker_sent_li.sort()
    partner_sent_li.sort()
    print("speaker, speaker_sent_li", speaker, speaker_sent_li)
    print("partner, partner_sent_li", partner, partner_sent_li)

    # dialogue_sent_li=speaker_sent_li+partner_sent_li
    # dialogue_sent_li.sort()

    speaker_adj_promixity=[]
    partner_adj_promixity=[]
    speaker_other_promixity=[]
    partner_other_promixity=[]

    for id, (speaker_sent, partner_sent)in enumerate(zip(speaker_sent_li, partner_sent_li)):
        # print("id, speaker_sent, partner_sent", id, speaker_sent, partner_sent)
        
        speaker_feature=session_speaker_df.loc[session_speaker_df['sentence']==speaker_sent][feature].item()
        partner_feature=session_partner_df.loc[session_partner_df['sentence']==partner_sent][feature].item()
        # print("speaker_feature, partner_feature", speaker_feature, partner_feature)

        adj_distance=abs(speaker_feature-partner_feature)

        sample_speaker_sent_li=speaker_sent_li.copy()
        sample_partner_sent_li=partner_sent_li.copy()
        sample_speaker_sent_li.remove(speaker_sent)
        sample_partner_sent_li.remove(partner_sent)
        
        random10_speaker_features=[session_speaker_df.loc[session_speaker_df['sentence']==sent][feature].item() for sent in random.sample(sample_speaker_sent_li, 10)]
        random10_partner_features=[session_partner_df.loc[session_partner_df['sentence']==sent][feature].item() for sent in random.sample(sample_partner_sent_li, 10)]
        # print("random10_speaker_features", random10_speaker_features)
        # print("random10_partner_features", random10_partner_features)
        
        other_speaker_distance=abs(speaker_feature-sum(random10_partner_features)/len(random10_partner_features))
        other_partner_distance=abs(partner_feature-sum(random10_speaker_features)/len(random10_speaker_features))

        speaker_adj_promixity.append(adj_distance)
        partner_adj_promixity.append(adj_distance)
        speaker_other_promixity.append(other_speaker_distance)
        partner_other_promixity.append(other_partner_distance)

        if id<len(speaker_sent_li)-1:
            next_speaker_sent=speaker_sent_li[id+1]
            # print("id, next_speaker_sent", id, next_speaker_sent)
            
            next_speaker_feature=session_speaker_df.loc[session_speaker_df['sentence']==next_speaker_sent][feature].item()
            next_adj_distance=abs(partner_feature-next_speaker_feature)
            
            next_sample_speaker_sent_li=sample_speaker_sent_li.copy()
            next_sample_speaker_sent_li.remove(next_speaker_sent)
            next_random10_speaker_features=[session_speaker_df.loc[session_speaker_df['sentence']==sent][feature].item() for sent in random.sample(next_sample_speaker_sent_li, 10)]
            # print("next_random10_speaker_features", next_random10_speaker_features)

            next_other_speaker_distance=abs(next_speaker_feature-sum(random10_partner_features)/len(random10_partner_features))
            next_other_partner_distance=abs(partner_feature-sum(next_random10_speaker_features)/len(next_random10_speaker_features))

            speaker_adj_promixity.append(next_adj_distance)
            partner_adj_promixity.append(next_adj_distance)
            speaker_other_promixity.append(next_other_speaker_distance)
            partner_other_promixity.append(next_other_partner_distance)
        

# return speaker_adj_promixity, partner_adj_promixity, speaker_other_promixity, partner_other_promixity

processing 7778
speaker, speaker_sent_li 77 [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79]
partner, partner_sent_li 78 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80]
processing 7172
speaker, speaker_sent_li 71 [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79]
partner, partner_sent_li 72 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80]
processing 6566
speaker, speaker_sent_li 65 [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79]
partner, partner_sent_li 66 [2

In [110]:
def compute_sentence_proximity(input_df, feature, session):
    session_input_df=input_df[input_df['session']==session]
    
    dyad_reg=r"(\d{2})(\d{2})"
    dyad_li=list(set(session_input_df['dyad'].tolist()))
    
    speaker_adj_promixity_dict={}
    partner_adj_promixity_dict={}
    speaker_other_promixity_dict={}
    partner_other_promixity_dict={}
    
    for dyad in dyad_li:
        print("processing", dyad)
        match = re.search(dyad_reg, str(dyad))
        speaker=int(match.group(1))
        partner=int(match.group(2))
    
        session_speaker_df=session_input_df[session_input_df['speaker']==speaker]
        session_partner_df=session_input_df[session_input_df['speaker']==partner]
    
        speaker_sent_li=session_speaker_df['sentence'].tolist()
        partner_sent_li=session_partner_df['sentence'].tolist()
        
        speaker_sent_li.sort()
        partner_sent_li.sort()
        print("speaker, speaker_sent_li", speaker, speaker_sent_li)
        print("partner, partner_sent_li", partner, partner_sent_li)
    
        # dialogue_sent_li=speaker_sent_li+partner_sent_li
        # dialogue_sent_li.sort()

        speaker_adj_promixity=speaker_adj_promixity_dict[dyad]=[]
        partner_adj_promixity=partner_adj_promixity_dict[dyad]=[]
        speaker_other_promixity=speaker_other_promixity_dict[dyad]=[]
        partner_other_promixity=partner_other_promixity_dict[dyad]=[]
    
        for id, (speaker_sent, partner_sent)in enumerate(zip(speaker_sent_li, partner_sent_li)):
            # print("id, speaker_sent, partner_sent", id, speaker_sent, partner_sent)
            
            speaker_feature=session_speaker_df.loc[session_speaker_df['sentence']==speaker_sent][feature].item()
            partner_feature=session_partner_df.loc[session_partner_df['sentence']==partner_sent][feature].item()
            # print("speaker_feature, partner_feature", speaker_feature, partner_feature)
    
            adj_distance=abs(speaker_feature-partner_feature)
    
            sample_speaker_sent_li=speaker_sent_li.copy()
            sample_partner_sent_li=partner_sent_li.copy()
            sample_speaker_sent_li.remove(speaker_sent)
            sample_partner_sent_li.remove(partner_sent)
            
            random10_speaker_features=[session_speaker_df.loc[session_speaker_df['sentence']==sent][feature].item() for sent in random.sample(sample_speaker_sent_li, 10)]
            random10_partner_features=[session_partner_df.loc[session_partner_df['sentence']==sent][feature].item() for sent in random.sample(sample_partner_sent_li, 10)]
            # print("random10_speaker_features", random10_speaker_features)
            # print("random10_partner_features", random10_partner_features)
            
            other_speaker_distance=abs(speaker_feature-sum(random10_partner_features)/len(random10_partner_features))
            other_partner_distance=abs(partner_feature-sum(random10_speaker_features)/len(random10_speaker_features))
    
            speaker_adj_promixity.append(adj_distance)
            partner_adj_promixity.append(adj_distance)
            speaker_other_promixity.append(other_speaker_distance)
            partner_other_promixity.append(other_partner_distance)
    
            if id<len(speaker_sent_li)-1:
                next_speaker_sent=speaker_sent_li[id+1]
                # print("id, next_speaker_sent", id, next_speaker_sent)
                
                next_speaker_feature=session_speaker_df.loc[session_speaker_df['sentence']==next_speaker_sent][feature].item()
                next_adj_distance=abs(partner_feature-next_speaker_feature)
                
                next_sample_speaker_sent_li=sample_speaker_sent_li.copy()
                next_sample_speaker_sent_li.remove(next_speaker_sent)
                next_random10_speaker_features=[session_speaker_df.loc[session_speaker_df['sentence']==sent][feature].item() for sent in random.sample(next_sample_speaker_sent_li, 10)]
                # print("next_random10_speaker_features", next_random10_speaker_features)
    
                next_other_speaker_distance=abs(next_speaker_feature-sum(random10_partner_features)/len(random10_partner_features))
                next_other_partner_distance=abs(partner_feature-sum(next_random10_speaker_features)/len(next_random10_speaker_features))
    
                speaker_adj_promixity.append(next_adj_distance)
                partner_adj_promixity.append(next_adj_distance)
                speaker_other_promixity.append(next_other_speaker_distance)
                partner_other_promixity.append(next_other_partner_distance)
            
    
    return speaker_adj_promixity_dict, partner_adj_promixity_dict, speaker_other_promixity_dict, partner_other_promixity_dict
        

In [117]:
speaker_adj_promixity_dict, partner_adj_promixity_dict, speaker_other_promixity_dict, partner_other_promixity_dict=compute_sentence_proximity(input_df, feature, session)
dyad_keys=list(speaker_adj_promixity_dict.keys())
print(dyad_keys)
print(len(speaker_adj_promixity_dict[7778]), speaker_adj_promixity_dict[7778])

processing 7778
speaker, speaker_sent_li 77 [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79]
partner, partner_sent_li 78 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80]
processing 7172
speaker, speaker_sent_li 71 [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79]
partner, partner_sent_li 72 [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80]
processing 6566
speaker, speaker_sent_li 65 [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79]
partner, partner_sent_li 66 [2

In [119]:
for k in dyad_keys:
    dyad_output_df=pd.DataFrame(list(zip(speaker_adj_promixity_dict[k], partner_adj_promixity_dict[k], speaker_other_promixity_dict[k], partner_other_promixity_dict[k])), 
                               columns=['speaker_adj_promixity', 'partner_adj_promixity', 'speaker_other_promixity', 'partner_other_promixity_dict'])
    dyad_output_df.to_csv("{}_promixity.csv".format(k))